In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('cars.csv')
df.head(5)

,brand,km_driven,fuel,owner,selling_price
0,Maruti,145500,Diesel,First Owner,450000
1,Skoda,120000,Diesel,Second Owner,370000
2,Honda,140000,Petrol,Third Owner,158000
3,Hyundai,127000,Diesel,First Owner,225000
4,Maruti,120000,Petrol,First Owner,130000


In [3]:
df['brand'].value_counts()  # TOO many brands, can't afford having that much extra columns

brand
Maruti           2448
Hyundai          1415
Mahindra          772
Tata              734
Toyota            488
Honda             467
Ford              397
Chevrolet         230
Renault           228
Volkswagen        186
BMW               120
Skoda             105
Nissan             81
Jaguar             71
Volvo              67
Datsun             65
Mercedes-Benz      54
Fiat               47
Audi               40
Lexus              34
Jeep               31
Mitsubishi         14
Force               6
Land                6
Isuzu               5
Kia                 4
Ambassador          4
Daewoo              3
MG                  3
Ashok               1
Opel                1
Peugeot             1
Name: count, dtype: int64

In [4]:
df['fuel'].value_counts()

fuel
Diesel    4402
Petrol    3631
CNG         57
LPG         38
Name: count, dtype: int64

In [5]:
df['owner'].value_counts()


owner
First Owner             5289
Second Owner            2105
Third Owner              555
Fourth & Above Owner     174
Test Drive Car             5
Name: count, dtype: int64

#### ONE HOT ENCODING USING PANDAS


In [32]:
# using OHE except on brand, as it has many categories
pd.get_dummies(df, columns=['fuel', 'owner'], dtype=np.int32)

,brand,km_driven,selling_price,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_First Owner,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,0,1,0,0,1,0,0,0,0
1,Skoda,120000,370000,0,1,0,0,0,0,1,0,0
2,Honda,140000,158000,0,0,0,1,0,0,0,0,1
3,Hyundai,127000,225000,0,1,0,0,1,0,0,0,0
4,Maruti,120000,130000,0,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai,110000,320000,0,0,0,1,1,0,0,0,0
8124,Hyundai,119000,135000,0,1,0,0,0,1,0,0,0
8125,Maruti,120000,382000,0,1,0,0,1,0,0,0,0
8126,Tata,25000,290000,0,1,0,0,1,0,0,0,0


#### K-1 OHE (Eliminate Multi-Collinearity)

In [31]:
# using OHE except on brand as it has many categories 
# and also eliminating first newly formed column of each feature 
# to reduce multicollinearity
pd.get_dummies(df, columns=['fuel', 'owner'], drop_first=True, dtype=np.int32)

,brand,km_driven,selling_price,fuel_Diesel,fuel_LPG,fuel_Petrol,owner_Fourth & Above Owner,owner_Second Owner,owner_Test Drive Car,owner_Third Owner
0,Maruti,145500,450000,1,0,0,0,0,0,0
1,Skoda,120000,370000,1,0,0,0,1,0,0
2,Honda,140000,158000,0,0,1,0,0,0,1
3,Hyundai,127000,225000,1,0,0,0,0,0,0
4,Maruti,120000,130000,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
8123,Hyundai,110000,320000,0,0,1,0,0,0,0
8124,Hyundai,119000,135000,1,0,0,1,0,0,0
8125,Maruti,120000,382000,1,0,0,0,0,0,0
8126,Tata,25000,290000,1,0,0,0,0,0,0


##### Don't use pandas for One Hot Encoding in ML projects
It doesn't remember which column it discarded to reduce multicollinearity

### One Hot Encoding using Sklearn

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:,0:4], df.iloc[:, -1], test_size=0.2, random_state=42)

In [9]:
X_train.head(5)

,brand,km_driven,fuel,owner
6518,Tata,2560,Petrol,First Owner
6144,Honda,80000,Petrol,Second Owner
6381,Hyundai,150000,Diesel,Fourth & Above Owner
438,Maruti,120000,Diesel,Second Owner
5939,Maruti,25000,Petrol,First Owner


In [10]:
from sklearn.preprocessing import OneHotEncoder

In [11]:
ohe = OneHotEncoder(drop='first', sparse_output=False)  # first to drop the first column formed to reduce multicollinearity

In [12]:
X_train_new = ohe.fit_transform(X_train[['fuel', 'owner']])

In [13]:
X_test_new = ohe.transform(X_test[['fuel', 'owner']])

In [14]:
# We have applied OHE on fuel and owner and
# now I would merge X_train_new with the first 2 columns of X_train
# I would now collect first 2 columns as numpy arrays and stack them 
# horizontally with the X_train_new

In [15]:
np.hstack((X_train[['brand', 'km_driven']].values, X_train_new))

array([['Tata', 2560, 0.0, ..., 0.0, 0.0, 0.0],
       ['Honda', 80000, 0.0, ..., 1.0, 0.0, 0.0],
       ['Hyundai', 150000, 1.0, ..., 0.0, 0.0, 0.0],
       ...,
       ['Hyundai', 35000, 0.0, ..., 0.0, 0.0, 0.0],
       ['Maruti', 27000, 1.0, ..., 0.0, 0.0, 0.0],
       ['Maruti', 70000, 0.0, ..., 1.0, 0.0, 0.0]], dtype=object)

#### OHE on brand
It has too many categories, perform One Hot Encoding on Frequent Top Brands and merge the rest into one feature

In [16]:
counts = df['brand'].value_counts()

In [17]:
df['brand'].nunique()
threshold = 100

In [18]:
brands = counts[counts <= threshold].index

In [19]:
brands

Index(['Nissan', 'Jaguar', 'Volvo', 'Datsun', 'Mercedes-Benz', 'Fiat', 'Audi',
       'Lexus', 'Jeep', 'Mitsubishi', 'Force', 'Land', 'Isuzu', 'Kia',
       'Ambassador', 'Daewoo', 'MG', 'Ashok', 'Opel', 'Peugeot'],
      dtype='object', name='brand')

In [26]:
df['brand']= df['brand'].replace(brands, 'Uncommon')

In [27]:
df['brand'].value_counts()

brand
Maruti        2448
Hyundai       1415
Mahindra       772
Tata           734
Uncommon       538
Toyota         488
Honda          467
Ford           397
Chevrolet      230
Renault        228
Volkswagen     186
BMW            120
Skoda          105
Name: count, dtype: int64

In [30]:
pd.get_dummies(df, columns=['brand'], drop_first=True, dtype=np.int32)

,km_driven,fuel,owner,selling_price,brand_Chevrolet,brand_Ford,brand_Honda,brand_Hyundai,brand_Mahindra,brand_Maruti,brand_Renault,brand_Skoda,brand_Tata,brand_Toyota,brand_Uncommon,brand_Volkswagen
0,145500,Diesel,First Owner,450000,0,0,0,0,0,1,0,0,0,0,0,0
1,120000,Diesel,Second Owner,370000,0,0,0,0,0,0,0,1,0,0,0,0
2,140000,Petrol,Third Owner,158000,0,0,1,0,0,0,0,0,0,0,0,0
3,127000,Diesel,First Owner,225000,0,0,0,1,0,0,0,0,0,0,0,0
4,120000,Petrol,First Owner,130000,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8123,110000,Petrol,First Owner,320000,0,0,0,1,0,0,0,0,0,0,0,0
8124,119000,Diesel,Fourth & Above Owner,135000,0,0,0,1,0,0,0,0,0,0,0,0
8125,120000,Diesel,First Owner,382000,0,0,0,0,0,1,0,0,0,0,0,0
8126,25000,Diesel,First Owner,290000,0,0,0,0,0,0,0,0,1,0,0,0
